In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [4]:
import math


In [5]:
class Value:
    def __init__(self,data,_children = (),_op='',label=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.label = label
        self.grad = 0.0
        self._backward = lambda:None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data + other.data,(self,other),'+')
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward =_backward
        return out
    
    def __mul__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        out = Value(self.data * other.data,(self,other),'*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __rmul__(self,other):
        return  self * other
    
    def __truediv__(self,other):
        return self * other**-1
    
    def __pow__(self,other):
        assert isinstance(other,(int,float)),"only supporting int/float"
        out = Value(self.data**other,(self, ),f'**{other}')
        
        def _backward():
            self.grad += other * self.data**(other-1) * out.grad
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x),(self, ),'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        return out
    
    def __sub__(self,other):
        return self+(-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x)-1)/(math.exp(2*x)+1)
        out = Value(t,(self,),'tanh')
        def _backward():
            self.grad += (1-t**2)*out.grad
        
        out._backward = _backward
        return out
    
    def backward(self):
        #topological sort
        topo = []
        visited =set()
        def buildTopo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    buildTopo(child)
                topo.append(v)
        buildTopo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [6]:
from graphviz import Digraph
def trace(root):
    nodes,edges = set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes,edges

def draw_dot(root):
    dot = Digraph(format ='svg',graph_attr={'rankdir':'LR'})
    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n))
        dot.node(name = uid,label = "{ %s | data %.4f | grad %.4f }" % (n.label,n.data,n.grad),shape='record')
        if(n._op):
            dot.node(name=uid+n._op,label=n._op)
            dot.edge(uid+n._op,uid)
    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2))+n2._op)
        
    return dot

In [7]:
import random

In [8]:
class Neuron:
    def __init__(self,nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
    def __call__(self,x):
        act = Value(0)
        arr = [wi*xi for wi,xi in zip(self.w,x)]+[self.b]
        for i,j in enumerate(arr):
            act = act + arr[i]
        out = act.tanh()
        return out
    def parameters(self):
        return self.w + [self.b]
    
    
class Layer:
    def __init__(self,nin,nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self,x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs)==1 else outs
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    def __init__(self,nin,nouts):
        sz = [nin]+nouts
        self.layers = [Layer(sz[i],sz[i+1]) for i in range(len(nouts))]
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    def parameters(self):
        return [p for layers in self.layers for p in layers.parameters()]

In [9]:
x = [2.0,3.0,-1.0]
n3 = MLP(3,[4,4,1])
len(n3.parameters())


41

In [12]:
def generateT(a):
    if a == '&' :
        xs = [
            [0.0,0.0],
            [0.0,1.0],
            [1.0,0.0],
            [1.0,1.0],
        ]
        ys = [0.0,1.0,1.0,0.0]
        return xs,ys
    elif a == '|' :
        xs = [
            [0.0,0.0],
            [0.0,1.0],
            [1.0,0.0],
            [1.0,1.0],
        ]
        ys = [0.0,1.0,1.0,0.0]
        return xs,ys
    elif a == '^' :
        xs = [
            [0.0,0.0],
            [0.0,1.0],
            [1.0,0.0],
            [1.0,1.0],
        ]
        ys = [0.0,1.0,1.0,0.0]
        return xs,ys
    else :
        return "Enter & | ^"

In [13]:
input1 = '&'
user_input = generateT(input1)
ypred = [n3(x) for x in user_input[0]]
arr = [(yout - ygt) ** 2 for ygt,yout in zip(user_input[1],ypred)]
loss = Value(0)
for i,j in enumerate(arr):
    loss = loss + arr[i]
print(loss)
print(ypred)
    

Value(data=1.8923079788564003)
[Value(data=-0.12685659848717715), Value(data=0.03868898696052555), Value(data=0.03793867328519178), Value(data=0.1628942053156771)]


In [ ]:
loss.backward()

In [ ]:
n3.layers[0].neurons[0].w[0].data

2.2422992491220475

In [ ]:
def modelfit(userinput,epochs):
    user_input = generateT(userinput)
    epochs = 100
    for i in range(epochs):
        ypred = [n3(x) for x in user_input[0]]
        arr = [(yout - ygt) ** 2 for ygt,yout in zip(user_input[1],ypred)]
        loss = Value(0)
        for i,j in enumerate(arr):
            loss = loss + arr[i]
        for p in n3.parameters():
            p.grad = 0.0
        loss.backward()
        for p in n3.parameters():
            p.data += -0.1*p.grad
        print(loss)

Value(data=1.2378710729825149)
Value(data=1.1887376928932594)
Value(data=1.0151659181558483)
Value(data=0.9840154086644087)
Value(data=0.9587444672194526)
Value(data=0.9363549251998382)
Value(data=0.9144083166805445)
Value(data=0.8922321681959703)
Value(data=0.8689582348203797)
Value(data=0.8444102484728585)
Value(data=0.8182045303929864)
Value(data=0.7903430063513939)
Value(data=0.7606416193651098)
Value(data=0.7293120114095492)
Value(data=0.6963024999907097)
Value(data=0.6624336739486745)
Value(data=0.6277283717782246)
Value(data=0.5956193446280553)
Value(data=0.5650147225927065)
Value(data=0.5536056372052721)
Value(data=0.5394922399069076)
Value(data=0.6159478624695984)
Value(data=0.5400933898491929)
Value(data=0.7185624756502944)
Value(data=0.4306832778548828)
Value(data=0.5388100082975009)
Value(data=0.42662686045404036)
Value(data=0.586866010603204)
Value(data=0.3483989074957109)
Value(data=0.45567218555066624)
Value(data=0.32643647700044576)
Value(data=0.4266067933667252)
Value(

In [ ]:
[n3(x) for x in xs]


[Value(data=0.004685942631777358),
 Value(data=0.9203000804015056),
 Value(data=0.9194518524159522),
 Value(data=0.00470811377868018)]